## Enviorment Setting

Both wandb and conda are optional to install. If you need it, please uncomment the following section:

In [1]:
# !conda create -n yolov8 python=3.7
# !conda activate yolov8
# !pip install wandb #wandb is a tool for experiment tracking, logging, and visualization
# !pip install ultralytics


# Call the required library files

In [1]:
import os
import shutil
import cv2
import random
import matplotlib.pyplot as plt
import numpy as np
import copy
from ultralytics import YOLO


input image size

In [2]:
image_size=640


# Creating Working Directories
Let's move the images and labels of all the classes to the working space and rename them according to their respective classes: buffalo, elephant, rhino, or zebra.

In [4]:
# Creating paths for separate images and labels
curr_path=os.getcwd()
imgtrainpath = os.path.join(curr_path,'images','train')
imgvalpath=os.path.join(curr_path,'images','validation')
imgtestpath=os.path.join(curr_path,'images','test')

labeltrainpath=os.path.join(curr_path,'labels','train')
labelvalpath=os.path.join(curr_path,'labels','validation')
labeltestpath=os.path.join(curr_path,'labels','test')


# 定义一个函数来清空或创建目录
def clear_or_create_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)  # 删除目录及其所有内容
    os.makedirs(path)  # 创建新的空目录

# 应用于所有路径
clear_or_create_dir(imgtrainpath)
clear_or_create_dir(imgvalpath)
clear_or_create_dir(imgtestpath)
clear_or_create_dir(labeltrainpath)
clear_or_create_dir(labelvalpath)
clear_or_create_dir(labeltestpath)


# # Creating directories for all paths defined
# os.makedirs(imgtrainpath)
# os.makedirs(imgvalpath)
# os.makedirs(imgtestpath)
# os.makedirs(labeltrainpath)
# os.makedirs(labelvalpath)
# os.makedirs(labeltestpath)

In [ ]:
# Defining data path and listing its contents
ip_datapath='output'
os.listdir(ip_datapath)

In [6]:
for dirname in os.listdir(ip_datapath):
    dirpath=os.path.join(ip_datapath, dirname)
    for file in os.listdir(dirpath):
        filepath=os.path.join(dirpath, file)
        newname=dirname+'_'+file
        if file.endswith((".txt")): # if label file, take it to label train path
            shutil.copy(filepath, labeltrainpath)
            path=os.path.join(labeltrainpath, file)
            newpath=os.path.join(labeltrainpath, newname)
        elif file.endswith((".jpg", ".JPG")): # if image file, resize and take it to image train path
            # img_resized=cv2.resize(cv2.imread(filepath), (image_size, image_size))
            img_resized=cv2.imread(filepath)  #这里删除了resize
            path=os.path.join(imgtrainpath, file)
            cv2.imwrite(path, img_resized)
            newpath=os.path.join(imgtrainpath, newname)
        os.rename(path, newpath) # Rename the file (label or image)

In [ ]:
# Checking the number of image and label files for all classes
len(os.listdir(imgtrainpath)), len(os.listdir(labeltrainpath))

# Extracting and Visualizing Test Images
We need to create a tough test set to see how well our model performs.

For the test set:

Choose images that have at least two different types of animals. For example, an image with a buffalo and a zebra is good for the test set.
Images can have more than two animals, like an image with two elephants and one buffalo.
For the training and validation sets:

Use images that have only one type of animal. For example, an image with just rhinos.
It's okay if there are multiple rhinos in one image, as long as there are no other types of animals.

In [8]:
# # function to check if all elements in the list are actually the same
# def are_all_elements_same(lst):
#     if not lst:
#         return True  # An empty list is considered to have all elements the same.

#     first_element = lst[0]
#     for element in lst[1:]:
#         if element != first_element:
#             return False

#     return True

Let's extract all the images with different types of animals present in the same image


In [9]:
# for file in os.listdir(labeltrainpath):
#     classes_list=[]
#     with open(os.path.join(labeltrainpath, file), "r") as f:
#         for line in f:
#             class_id,_,_,_,_=line.strip().split(" ")
#             classes_list.append(class_id) # creating list of all unique animal types in given image
            
#     # Checking if different types of animals are present in image
#     if not are_all_elements_same(classes_list): 
#         filepath=os.path.join(labeltrainpath, file)
#         newpath=os.path.join(labeltestpath, file)
#         shutil.move(filepath, newpath) # moving label file to test path
#         basename=os.path.splitext(file)[0]
#         print(basename) # printing the image name
#         imgfilename=basename+'.jpg'
#         oldimgfilepath=os.path.join(imgtrainpath, imgfilename)
#         newimgfilepath=os.path.join(imgtestpath, imgfilename) 
#         shutil.move(oldimgfilepath, newimgfilepath) # moving image to test path

The above printed images are all those which have more than one type of animal in it, although the name suggests that there is just one animal. Let's check few of these images

In [10]:
# import os
# import shutil

# source_path = '/home/dfrobot/yolo/yolov8_train/images/test'
# destination_path = '/home/dfrobot/yolo/yolov8_sdimg2img_train/images/test'

# # 检查目标路径是否存在
# if not os.path.exists(destination_path):
#     # 如果不存在，则直接拷贝整个目录树
#     shutil.copytree(source_path, destination_path)
# else:
#     # 如果目标路径已存在，遍历源目录中的文件和目录
#     for item in os.listdir(source_path):
#         source_item = os.path.join(source_path, item)
#         destination_item = os.path.join(destination_path, item)
        
#         # 如果是文件且目标路径中不存在，则拷贝文件
#         if os.path.isfile(source_item) and not os.path.exists(destination_item):
#             shutil.copy2(source_item, destination_item)
#         # 如果是目录且目标路径中不存在，则递归拷贝整个目录树
#         elif os.path.isdir(source_item) and not os.path.exists(destination_item):
#             shutil.copytree(source_item, destination_item)


# source_path_lab = '/home/dfrobot/yolo/yolov8_train/labels/test'
# destination_path_lab = '/home/dfrobot/yolo/yolov8_sdimg2img_train/labels/test'

# # 检查目标路径是否存在
# if not os.path.exists(destination_path_lab):
#     # 如果不存在，则直接拷贝整个目录树
#     shutil.copytree(source_path_lab, destination_path_lab)
# else:
#     # 如果目标路径已存在，遍历源目录中的文件和目录
#     for item in os.listdir(source_path_lab):
#         source_item = os.path.join(source_path_lab, item)
#         destination_item = os.path.join(destination_path_lab, item)
        
#         # 如果是文件且目标路径中不存在，则拷贝文件
#         if os.path.isfile(source_item) and not os.path.exists(destination_item):
#             shutil.copy2(source_item, destination_item)
#         # 如果是目录且目标路径中不存在，则递归拷贝整个目录树
#         elif os.path.isdir(source_item) and not os.path.exists(destination_item):
#             shutil.copytree(source_item, destination_item)

In [11]:
# plt.figure(figsize=(30,30))
# for i in range(6):
#     test_image=os.path.join(imgtestpath, os.listdir(imgtestpath)[i])
#     ax=plt.subplot(3,2,i+1)
    
#     # Display actual image
#     plt.imshow(cv2.imread(test_image)) 
#     plt.xticks([])
#     plt.yticks([])

In [12]:
# # Checking the size of test dataset
# len(os.listdir(imgtestpath)), len(os.listdir(labeltestpath))


In [13]:
# # Checking the size of training(+validation) dataset
# len(os.listdir(imgtrainpath)), len(os.listdir(labeltrainpath))

## Separating Training and Validation Dataset

Let's now partition the training dataset into separate training and validation datasets.

In [14]:
# moving 20% of data to validation

factor=0.2 

for file in random.sample(os.listdir(imgtrainpath), int(len(os.listdir(imgtrainpath))*factor)):
    basename=os.path.splitext(file)[0]
    textfilename=basename+'.txt'
    labelfilepath=os.path.join(labeltrainpath, textfilename)
    labeldestpath=os.path.join(labelvalpath, textfilename)
    imgfilepath=os.path.join(imgtrainpath, file)
    imgdestpath=os.path.join(imgvalpath, file)
    shutil.move(imgfilepath, imgdestpath)
    shutil.move(labelfilepath, labeldestpath)

In [ ]:
# Checking the size of training dataset
len(os.listdir(imgtrainpath)), len(os.listdir(labeltrainpath))

In [ ]:
# Checking the size of validation dataset
len(os.listdir(imgvalpath)), len(os.listdir(labelvalpath))

Now, we need to write a function to extract the bounding box coordinates from the label files. These label files provide details such as the image type, the coordinates of the image center, and the dimensions of the image.

In [17]:
# # function to obtain bounding box coordinates from text label files
# def get_bbox_from_label(text_file_path):
#     bbox_list=[]
#     with open(text_file_path, "r") as file:
#         for line in file:
#             class_id,x_centre,y_centre,width,height=line.strip().split(" ")
#             x1=(float(x_centre)+(float(width)/2))*image_size
#             x0=(float(x_centre)-(float(width)/2))*image_size
#             y1=(float(y_centre)+(float(height)/2))*image_size
#             y0=(float(y_centre)-(float(height)/2))*image_size
            
#             vertices=np.array([[int(x0), int(y0)], [int(x1), int(y0)], 
#                                [int(x1),int(y1)], [int(x0),int(y1)]])
#             bbox_list.append(vertices)      
#     return tuple(bbox_list)

In [18]:
# # defining red color in RGB to draw bounding box
# red=(255,0,0) 

In [19]:
# # Drawing bounding box for random images in training data
# plt.figure(figsize=(30,30))
# for i in range(1,8,2):
#     k=random.randint(0, len(os.listdir(imgtrainpath))-1)
#     img_path=os.path.join(imgtrainpath, sorted(os.listdir(imgtrainpath))[k])
#     label_path=os.path.join(labeltrainpath, sorted(os.listdir(labeltrainpath))[k])
#     bbox=get_bbox_from_label(label_path) # extracting bounding box coordinates
#     image=cv2.imread(img_path)
#     image_copy=copy.deepcopy(image)
#     ax=plt.subplot(4, 2, i)
#     plt.imshow(image) # displaying image
#     plt.xticks([])
#     plt.yticks([])
#     cv2.drawContours(image_copy, bbox, -1, red, 2) # drawing bounding box on copy of image
#     ax=plt.subplot(4, 2, i+1)
#     plt.imshow(image_copy) # displaying image with bounding box
#     plt.xticks([])
#     plt.yticks([])

## Creating config file

In [20]:
# defining newline variable for config file
newline='\n'

In [21]:
# Starting with a comment in config file
ln_1='# Train/val/test sets'+newline

# train, val and test path declaration
ln_2='train: ' +"'"+imgtrainpath+"'"+newline
ln_3='val: ' +"'" + imgvalpath+"'"+newline
ln_4='test: ' +"'" + imgtestpath+"'"+newline
ln_5=newline
ln_6='# Classes'+newline

# names of the classes declaration
ln_7='names:'+newline
ln_8 = '  0: base'+newline
ln_9 = '  1: block_green'+newline
ln_10 = '  2: block_purple'+newline
ln_11 = '  3: block_red'+newline
ln_12 = '  4: block_yellow'+newline
ln_13 = '  5: hand_0'+newline
ln_14 = '  6: hand_1'+newline
ln_15 = '  7: hand_2'+newline
ln_16 = '  8: hand_3'+newline
ln_17 = '  9: hand_4'+newline
ln_18 = '  10: hand_5'+newline
ln_19 = '  11: people_man'+newline
ln_20 = '  12: people_police'+newline
ln_21 = '  13: people_woman'+newline
ln_22 = '  14: sign_i10'+newline
ln_23 = '  15: sign_i12'+newline
ln_24 = '  16: sign_i3'+newline
ln_25 = '  17: sign_i9'+newline
ln_26 = '  18: sign_pl40'+newline
ln_27 = '  19: sign_pr40'+newline
ln_28 = '  20: sign_ps'+newline
ln_29 = '  21: sign_w48'+newline
ln_30 = '  22: sign_w65'+newline
ln_31 = '  23: traffic_green'+newline
ln_32 = '  24: traffic_none'+newline
ln_33 = '  25: traffic_red'+newline



# ln_12='lr: 0.001'  # 调整学习率


# config_lines=[ln_1, ln_2, ln_3, ln_4, ln_5, ln_6, ln_7, ln_8, ln_9, ln_10, ln_11, ln_12, ln_13, ln_14, ln_15, ln_16]
config_lines=[ln_1, ln_2, ln_3, ln_4, ln_5, ln_6, ln_7, ln_8, ln_9, ln_10, ln_11, ln_12, ln_13, ln_14, ln_15, ln_16, ln_17, ln_18, ln_19, ln_20, ln_21, ln_22, ln_23, ln_24, ln_25, ln_26, ln_27, ln_28, ln_29, ln_30, ln_31, ln_32, ln_33]

In [ ]:
# Creating path for config file
config_path=os.path.join(curr_path, 'config.yaml')
config_path

In [23]:
# Writing config file
with open(config_path, 'w') as f:
    f.writelines(config_lines)

## Model training

In [ ]:
# Using YOLO's pretrained model architecture and weights for training
model=YOLO('yolov8n.yaml').load('yolov8n.pt')

In [ ]:
# Training the model
# results=model.train(data=config_path, epochs=40, iou=0.5, conf=0.01, imgsz=640)
results=model.train(data=config_path, epochs=100, iou=0.5, conf=0.01, imgsz=640, save_period=1)

